In [87]:
# Importation des modules

import pandas as pd
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
#Importation des données

df = pd.read_csv('data/bdd_complete2.csv', sep = ',', encoding = 'latin-1')

In [89]:
# Echantillon de la base
print(df.shape)
df.sample(5)

(499843, 26)


,Unnamed: 0,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor
133154,133155,emmanuelle menard,2019-03-15,Croissance et transformation des entreprises,Présidence de Mme Annie Genevard,Discussion des articles,Javais déposé lors de la première lecture du...,NaN,Présidence de Mme Annie Genevard,Ménard,F,49,NI,affaires économiques,0,False,0,1 - Soc. civile,6  Exp. Pol.,Master,1968,False,cadsupprive,133155,Exp,2019-03-01
264960,264961,jean paul lecoq,2018-12-17,Projet de loi de finances pour 2019,Présidence de M. Marc Le Fur,NaN,Nous vous lavons dit en première lecture mai...,(Applaudissements sur les bancs du groupe GDR.),Présidence de M. Marc Le Fur,Lecoq,M,59,GDR,Aff. Etrangeres,1,False,28,5 - Dép. sortant,5 - Dép. sortant,Pro,1958,False,EmployeInter,264961,Exp,2018-12-01
396444,396445,pierre yves bournazel,2017-11-02,Projet de loi de finances pour 2018,Présidence de Mme Carole Bureau-Bonnard,Seconde partie,Enjeu de société majeur toute politique de lu...,NaN,Présidence de Mme Carole Bureau-Bonnard,Bournazel,M,40,LC,Aff. Cult. & Educ.,0,True,12,4 - Entourage,4 - Entourage,ScPoProv,1977,True,collab,396445,Exp,2017-11-01
354638,354639,olivier faure,2020-03-03,Motions de censure,Présidence de Mme Annie Genevard,Explications de vote,Votre financement parce quil repose sur le r...,NaN,Présidence de Mme Annie Genevard,Faure,M,49,NG,Defense,1,True,21,5 - Dép. sortant,5 - Dép. sortant,Master,1968,False,collab,354639,Exp,2020-03-01
9232,9233,alain perea,2018-06-01,Évolution du logement de laménagement et du ...,Présidence de Mme Carole Bureau-Bonnard,Discussion des articles,Je défends en même temps mon amendement de rep...,NaN,Présidence de Mme Carole Bureau-Bonnard,Perea,M,46,LREM,DvptDurable,0,False,9,2 - 2de carrière,2 - 2de carrière,Master,1971,True,cadsuppub,9233,Exp,2018-06-01


In [90]:
# L'objectif est de nettoyer le texte dit par les intervenants. On s'intéresse donc à la colonne 'Réplique'

# Première approche : échantillon de 10 000 interventions

df_replique = df[['Réplique']]

In [91]:
df_replique.sample(10)

,Réplique
183531,La parole est à M. Joaquim Pueyo pour souten...
361622,Le contenu de la résolution et des débats auxq...
134813,Or vous le savez sur 1<U+0080>000 euros de d...
479796,On attend<U+0080>!
12812,Pour quelles raisons<U+0080>? Vous avez le dr...
353794,Dans le monde entier on nous regarde. Que dit-...
350598,rapporteur. Vous faites la liste de toutes les...
482190,Oui 8<U+0080>000 logements<U+0080>!
458390,Je me fonde sur les bribes dinformations que ...
29640,Cest la voix des lobbies<U+0080>!


In [92]:
# Première correction : les apostrophes sont codées par '\x92'. On les retire. 
# On supprime aussi les codes '<U+0080>' qui sont avant les points d'exclamation
# On remplace les tirets par des espaces
# On met tout en minuscule

df_replique['Réplique'] = df_replique.apply(
    lambda row: row.Réplique.replace('\x92', '').replace('<U+0080>', '').replace('-', ' ').lower(),
    axis = 1)


/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [93]:
# On tokenize les chaines de caractère

df_replique['tokenized'] = df_replique.apply(lambda row: nltk.word_tokenize(row.Réplique, language='french'), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [94]:
# On obtient bien des listes de mots

df_replique.head(10)

,Réplique,tokenized
0,si vous y donnez un avis favorable madame la ...,"[si, vous, y, donnez, un, avis, favorable, mad..."
1,plus généralement je soutiendrai un autre ame...,"[plus, généralement, je, soutiendrai, un, autr..."
2,ma question sadresse à m. françois de rugy m...,"[ma, question, sadresse, à, m., françois, de, ..."
3,le 1,"[le, 1]"
4,cette autorisation strictement encadrée par l...,"[cette, autorisation, strictement, encadrée, p..."
5,je suis également préoccupé par la situation d...,"[je, suis, également, préoccupé, par, la, situ..."
6,sagissant du cerema force est de constater qu...,"[sagissant, du, cerema, force, est, de, consta..."
7,pouvez vous exposer les nouvelles perspective...,"[pouvez, vous, exposer, les, nouvelles, perspe..."
8,cette initiative qui associera des citoyens à...,"[cette, initiative, qui, associera, des, citoy..."
9,la réponse est non. la proposition de loi que...,"[la, réponse, est, non, ., la, proposition, de..."


In [95]:
# On retire la ponctuation pour le moment

df_replique['tokenized'] = df_replique.apply(lambda row: [word for word in row.tokenized if word.isalpha()], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [96]:
# On retire les stopwords

stop_words = set(stopwords.words('french'))

df_replique['tokenized'] = df_replique.apply(lambda row: [word for word in row.tokenized if not word in stop_words], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [97]:
# Stemming

df_replique['stemmed'] = df_replique.apply(lambda row: [stemmer.stem(word) for word in row.tokenized], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [98]:
df_cleaned = df.copy()
df_cleaned['tokenized'] = df_replique['tokenized']
df_cleaned['stemmed'] = df_replique['stemmed']

In [100]:
df_cleaned.to_csv('data/data_cleaned.csv', sep = ',', encoding = 'latin-1')